# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [54]:
import pandas as pd

df = pd.read_csv("data/forestfires.csv")
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [55]:
df.drop(columns=["FFMC", "DMC", "DC", "ISI"])

,X,Y,month,day,temp,RH,wind,rain,area
0,7,5,mar,fri,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,25.6,42,4.0,0.0,0.00


In [56]:
df["fire"] = (df["area"] > 0).astype(int)     # 1 = fire
df.head()


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,fire
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0


In [57]:
month2num = dict(jan=1, feb=2, mar=3, apr=4, may=5, jun=6,
                 jul=7, aug=8, sep=9, oct=10, nov=11, dec=12)

df["mon"] = df["month"].str.lower().map(month2num)

import pandas as pd
import numpy as np

df["time"] = pd.to_datetime(
    dict(year = 2000,
         month = df["month"].str.lower().map(month2num),
         day = 15)
)

In [58]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,fire,mon,time
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0,3,2000-03-15
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,10,2000-10-15
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,10,2000-10-15
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0,3,2000-03-15
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,3,2000-03-15


In [59]:
import xarray as xr

ds = xr.Dataset(
    {
        "tas":     ("time", df["temp"].values),   # air temperature
        "hurs":    ("time", df["RH"].values),     # relative humidity
        "sfcWind": ("time", df["wind"].values),   # 10-m wind speed
        "pr":      ("time", df["rain"].values),   # 24-h rainfall
    },
    coords = {"time": df["time"]},
)

# --- attach units in pint-friendly spelling --------------------------------------
ds["tas"    ].attrs["units"] = "degC"
ds["hurs"   ].attrs["units"] = "%"
ds["sfcWind"].attrs["units"] = "km/h"
ds["pr"     ].attrs["units"] = "mm/day"

In [60]:
LAT = 41.79                                  # replace with lat from device
ds = ds.assign_coords(lat=LAT)          #  <— scalar, no 'time' dim
ds["lat"].attrs["units"] = "degrees_north"



# ----------------------------------------------------------
# 5.  Quick sanity-check: print every units attr
# ----------------------------------------------------------
for var in ["tas", "hurs", "sfcWind", "pr", "lat"]:
    print(f"{var:8s}  units → {ds[var].attrs.get('units')}")
    
print(ds.lat, ds.lat.dims)   # should show ()  ->  dimensionless

tas       units → degC
hurs      units → %
sfcWind   units → km/h
pr        units → mm/day
lat       units → degrees_north
<xarray.DataArray 'lat' ()> Size: 8B
array(41.79)
Coordinates:
    lat      float64 8B 41.79
Attributes:
    units:    degrees_north ()


In [64]:
import numpy as np
from xclim.indices.fire import cffwis_indices

fwi_ds = cffwis_indices(
    tas     = ds.tas,
    hurs    = ds.hurs,
    sfcWind = ds.sfcWind,
    pr      = ds.pr,
    lat     = ds.lat         
)

# df[["FFMC", "DMC", "DC", "ISI"]] = (
#     fwi_ds[["ffmc", "dmc", "dc", "isi"]]
#       .to_dataframe()
#       .reset_index(drop=True)
# )
# print(df.head())                # quick sanity-check
print("type =", type(fwi_ds))          # should show <class 'xarray.core.dataset.Dataset'>
print("is tuple?", isinstance(fwi_ds, tuple))

df[["FFMC", "DMC", "DC", "ISI"]] = (
    fwi_ds[["ffmc", "dmc", "dc", "isi"]]
      .to_dataframe()
      .reset_index(drop=True)
)

type = <class 'tuple'>
is tuple? True


TypeError: tuple indices must be integers or slices, not list

In [65]:
tmp = fwi_ds        # tmp is something like  ( <xarray.Dataset> , )

# extract the first (and only) element
fwi_ds = tmp[0]     # now fwi_ds is an xarray.Dataset

In [66]:
print("type =", type(fwi_ds))          # should show <class 'xarray.core.dataset.Dataset'>
print("is tuple?", isinstance(fwi_ds, tuple))

df[["FFMC", "DMC", "DC", "ISI"]] = (
    fwi_ds[["ffmc", "dmc", "dc", "isi"]]
      .to_dataframe()
      .reset_index(drop=True)
)

type = <class 'xarray.core.dataarray.DataArray'>
is tuple? False


ValueError: Error parsing datetime string "ffmc" at position 0

In [67]:
# -- call the function and keep the Dataset ----------------
fwi_ds = cffwis_indices(
    tas     = ds.tas,
    hurs    = ds.hurs,
    sfcWind = ds.sfcWind,
    pr      = ds.pr,
    lat     = ds.lat
)   #  ◀─ nothing after this parenthesis!

# -- sanity-check ------------------------------------------
assert isinstance(fwi_ds, xr.Dataset), "fwi_ds is not a Dataset!"

# -- merge the four codes into the original table ----------
df[["FFMC", "DMC", "DC", "ISI"]] = (
    fwi_ds[["ffmc", "dmc", "dc", "isi"]]
        .to_dataframe()
        .reset_index(drop=True)
)

print(df.head())

AssertionError: fwi_ds is not a Dataset!